In [1]:
import mujoco_py
from mujoco_py import load_model_from_path, MjSim, MjViewer
import os
import glfw
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.distributions import MultivariateNormal
from torch.distributions import Beta
from torch.optim import Adam
from collections import deque
import time
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import pickle
device = torch.device("cpu")

## network

In [2]:
class PolicyNN(nn.Module):
    def __init__(self, in_dim, out_dim, device, distribution):
        super(PolicyNN, self).__init__()

        self.layer1 = nn.Linear(in_dim, 512)
        self.layer2 = nn.Linear(512, 256)
        self.layer3 = nn.Linear(256, out_dim)
        self.device = device
        self.distribution = distribution

    def forward(self, obs):
        if isinstance(obs, np.ndarray):
            obs = torch.tensor(obs, dtype=torch.float).to(self.device)

        activation1 = F.relu(self.layer1(obs))
        activation2 = F.relu(self.layer2(activation1))
        out = self.layer3(activation2)
        if self.distribution == 'Normal': output = out
        elif self.distribution == 'Beta': output = F.softplus(out)

        return output

In [3]:
class PolicyNN(nn.Module):
    def __init__(self, in_dim, out_dim, device, distribution):
        super(PolicyNN, self).__init__()

        self.layer1 = nn.Linear(in_dim, 100)
        self.layer2 = nn.Linear(100, 50)
        self.layer3 = nn.Linear(50, 25)
        self.layer4 = nn.Linear(25, out_dim)
        self.device = device
        self.distribution = distribution

    def forward(self, obs):
        if isinstance(obs, np.ndarray):
            obs = torch.tensor(obs, dtype=torch.float).to(self.device)

        activation1 = F.relu(self.layer1(obs))
        activation2 = F.relu(self.layer2(activation1))
        activation3 = F.relu(self.layer3(activation2))
        out = self.layer4(activation3)
        if self.distribution == 'Normal': output = out
        elif self.distribution == 'Beta': output = F.softplus(out)

        return output

In [ ]:
class Model(nn.Module):
    def __init__(self, ):
        super()

## compute action

In [3]:
def get_action(actor, obs, cov_mat, distribution, action_range):
    if distribution == 'Normal':        
        mean = actor(obs)
        dist = MultivariateNormal(mean, cov_mat)
        action = dist.sample()
        log_prob = dist.log_prob(action)
    elif distribution == 'Beta':
        alpha_beta = actor(obs)
        alpha = alpha_beta[0: int(alpha_beta.shape[0]/2)] + 1
        beta = alpha_beta[int(alpha_beta.shape[0]/2): int(alpha_beta.shape[0])] + 1
        dist = Beta(alpha, beta)
        action0_1 = dist.sample()
        # to map [0, 1] to [act_minrange, act_maxrange]:
        action = (torch.tensor(action_range[1]) - torch.tensor(action_range[0])
                 ) * action0_1 + torch.tensor(action_range[0])
        log_prob = dist.log_prob(action0_1).mean()
    return action.detach().cpu().numpy(), log_prob    

## compute Q(s|a)

In [4]:
def compute_rtgs(batch_rews, gamma, device):
    batch_rtgs = []

    for ep_rews in reversed(batch_rews):
        discounted_reward = 0 

        for rew in reversed(ep_rews):
            discounted_reward = rew + discounted_reward * gamma
            batch_rtgs.insert(0, discounted_reward)

    batch_rtgs = torch.tensor(batch_rtgs, dtype=torch.float).to(device)
    return batch_rtgs

## evaluate V(S)

In [5]:
def evaluate(actor, critic, cov_mat, batch_obs, batch_acts, distribution, action_range):
    V = critic(batch_obs).squeeze()
    # This segment of code is similar to that in get_action()
    if distribution == 'Normal':
        mean = actor(batch_obs)
        dist = MultivariateNormal(mean, cov_mat)
        log_probs = dist.log_prob(batch_acts)
    elif distribution == 'Beta':
        alpha_beta = actor(batch_obs)
        alpha = alpha_beta[:, 0: int(alpha_beta.shape[1]/2)] + 1
        beta = alpha_beta[:, int(alpha_beta.shape[1]/2): int(alpha_beta.shape[1])] + 1
        dist = Beta(alpha , beta)
        #to remapmap [act_minrange, act_maxrange] to [0, 1]
        max_min = torch.tensor(action_range[1]) - torch.tensor(action_range[0])
        minn = torch.tensor(action_range[0])                     
        batch_acts0_1 = (batch_acts - minn)*(1-2e-5)/max_min + 1e-5 # to avoind beyound [0,1]
        log_probs = dist.log_prob(batch_acts0_1).mean(dim=1)
        
    return V, log_probs

## get reward

In [6]:
def euler_from_quaternion(quaternion):
    w = quaternion[0]; x = quaternion[1]; y = quaternion[2]; z = quaternion[3]
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    roll_x = np.arctan2(t0, t1)

    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    pitch_y = np.arcsin(t2)

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    yaw_z = np.arctan2(t3, t4)

    return roll_x, pitch_y, yaw_z 

def compute_reward(e_i, alpha_i):
    return np.exp(-alpha_i * e_i**2)

def get_reward(alpha, COM, COM_vel, quaternion, support_center, fall,
               w1 = 1.0/8,    w2 = 1.5/8,     w3 = 1/8,
               w4 = 0.5/8,    w5 = 0.5/8,    w6 = 0.5/8,
               w7 = 1.5/8,    w8 = 1.5/8):

    e_i = abs(support_center[0]-0.015 - COM[0])
    r_xCOM = compute_reward(e_i, alpha[0])
    
    e_i = abs(support_center[1] - COM[1])
    r_yCOM = compute_reward(e_i, alpha[1])
    
    e_i = abs(0.47 - COM[2])
    r_zCOM = compute_reward(e_i, alpha[2])

    e_i = COM_vel[0];    r_xdotCOM = compute_reward(e_i, alpha[3])
    e_i = COM_vel[1];    r_ydotCOM = compute_reward(e_i, alpha[3])
    e_i = COM_vel[2];    r_zdotCOM = compute_reward(e_i, alpha[3])
    
    roll_x, pitch_y, yaw_z = euler_from_quaternion(quaternion)
    e_i = roll_x;     r_roll  = compute_reward(e_i, alpha[4])
    e_i = pitch_y;    r_pitch = compute_reward(e_i, alpha[4])
    
    if not fall:
        total_reward = (w1 * r_xCOM + w2 * r_yCOM + w3 * r_zCOM +
                        w4 * r_xdotCOM + w5 * r_ydotCOM + w6 * r_zdotCOM +
                        w7 * r_roll + w8 * r_pitch)
    else:
        total_reward = 0
    return  total_reward

## initial condition

In [7]:
def initial_condition(sim, model, kp, kd, ki, mode, ball):    
    if mode == 'test': viewer = MjViewer(sim)
    if not ball:
        hip_torque = 1.0*np.random.uniform(low=1.0, high=4.2)
        knee_torque = 1.0*np.random.uniform(low=1.4, high=4.6)
        ankle_torque = 1.0*np.random.uniform(low=1.0, high=4.2)
        sim_time = np.random.uniform(low=60, high=120)
        for _ in range(int(sim_time)):
            sim.data.ctrl[:] = [0, hip_torque, knee_torque, ankle_torque, 0,
                                0, hip_torque, knee_torque, ankle_torque, 0]
            sim.step()
            if mode == 'test':
                viewer.render()
                time.sleep(0.01)

    elif ball:
        sim_time = np.random.uniform(low=140, high=180)
        shoot_impulse = np.random.uniform(low=2.5, high=3)
        shoot_angle = np.random.uniform(low=30, high=45)*np.pi/180
        shoot_duration = sim_time * model.opt.timestep
        shoot_force = shoot_impulse/shoot_duration
        external_push_x = -shoot_force * np.cos(shoot_angle)
        external_push_z =  shoot_force * np.sin(shoot_angle)
        qpos_d = np.array([0]*10)
        qvel_d = np.array([0]*10)
        int_pos = np.zeros(10)
        old_joints_position = [0]*10
        
        sim.data.xfrc_applied[24][:3] = [external_push_x, 0, external_push_z]
        sim_state = sim.get_state()
        joints_position = sim_state.qpos[7:17]
        joints_velocity = sim_state.qvel[6:16]
        for i in range(int(sim_time)):
            int_pos += model.opt.timestep * (( np.array(joints_position)
                                              +np.array(old_joints_position)) / 2-qpos_d)
            error = qpos_d - joints_position
            derror = qvel_d - joints_velocity
            apllied_torques = kp * error + kd * derror -ki *int_pos
            sim.data.ctrl[:] = apllied_torques
            #if i > sim_time - 2: sim.data.ctrl[:] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
            sim.step()
            if mode == 'test':
                viewer.render()
                time.sleep(0.01)
            old_joints_position = joints_position

            sim_state = sim.get_state()
            joints_position = sim_state.qpos[7:17]
            joints_velocity = sim_state.qvel[6:16]
    
    sim.data.time = 0
    sim.data.ctrl[:] = 0
    sim.data.xfrc_applied[:] = 0
    '''sim_state = sim.get_state()
    sim_state.qvel[:] = np.zeros(18)
    sim.set_state(sim_state)'''
    if mode == 'test': glfw.destroy_window(viewer.window)
    return sim


## Contact forces

In [8]:
def contact_force(sim):
    #contact_forces = np.zeros(8, dtype=np.float64)
    #contact_forces = np.zeros(2, dtype=np.float64)
    left_contact = []
    right_contact = []
    for i in range(sim.data.ncon):
        c_array = np.zeros(6, dtype=np.float64)
        contact = sim.data.contact[i]
        if contact.geom1 == 0 and contact.geom2 == 10:
            mujoco_py.functions.mj_contactForce(sim.model, sim.data, i, c_array)
            left_contact.append(c_array[0])
        elif contact.geom1 == 0 and contact.geom2 == 19:
            mujoco_py.functions.mj_contactForce(sim.model, sim.data, i, c_array)
            right_contact.append(c_array[0])
    
    '''if len(left_contact) == 4:
        contact_forces[0:4] = left_contact
    else:
        left_sensor = np.array([sim.data.sensordata[8], sim.data.sensordata[11],
                                sim.data.sensordata[14] ,sim.data.sensordata[17]])
        l_s_sort_indx = left_sensor.argsort()
        l_contact_sort = sorted(left_contact,reverse=True)
        for i in range(len(left_contact)):
            contact_forces[l_s_sort_indx[i]] = l_contact_sort[i]
    
    if len(right_contact) == 4:
        contact_forces[4:8] = right_contact
    else:
        right_sensor = np.array([sim.data.sensordata[20], sim.data.sensordata[23],
                                 sim.data.sensordata[26], sim.data.sensordata[29]])
        r_s_sort_indx = right_sensor.argsort()
        r_contact_sort = sorted(right_contact,reverse=True)
        for i in range(len(right_contact)):
            contact_forces[r_s_sort_indx[i] + 4] = r_contact_sort[i]
    return contact_forces'''
    #contact_forces[0] = sum(left_contact)
    #contact_forces[1] = sum(right_contact)
    return left_contact, right_contact

## PID step

In [9]:
def pid_step(sim, model, pid_time, kp, kd, ki, qpos_d, state_range, mode, 
             qvel_d = np.array([0]*10), int_pos = np.zeros(10), old_joints_position = [0]*10,
             old_left_contact = [], old_right_contact =[]):
    if mode == 'test': viewer = MjViewer(sim)
    sim_state = sim.get_state()
    joints_position = sim_state.qpos[7:17]
    joints_velocity = sim_state.qvel[6:16]
    fall = False
    for step in range(int(pid_time / model.opt.timestep)):
        int_pos += model.opt.timestep * (( np.array(joints_position)
                                          +np.array(old_joints_position)) / 2-qpos_d)
        error = qpos_d - joints_position
        derror = qvel_d - joints_velocity
        applied_torques = kp * error + kd * derror -ki *int_pos
        sim.data.ctrl[:] = applied_torques
        
        sim.step()
        if mode == 'test':
            viewer.render()
            time.sleep(0.002)
        old_joints_position = joints_position
        
        sim_state = sim.get_state()
        joints_position = sim_state.qpos[7:17]
        joints_velocity = sim_state.qvel[6:16]
        
        contact0 = sim.data.contact[0]
        geom_1 = sim.model.geom_id2name(contact0.geom1)
        geom_2 = sim.model.geom_id2name(contact0.geom2)
        '''
        if (sim.data.ncon > 0 and geom_1 == 'floor' and \
            geom_2 not in ('l_foot_link', 'r_foot_link', 'small_ball',
                           'l_ank_roll_link', 'r_ank_roll_link')) or\
        sim.data.sensordata[2]> 0.6 :
            fall = True
            break
        '''
        left_contact, right_contact = contact_force(sim)
        if sim.data.ncon > 0 and ((step>35 and 
                                  (len(left_contact)<2 or len(right_contact)<2)) or
                                  (geom_1 == 'floor' and geom_2 not in ('l_foot_link', 'r_foot_link',
                                                                        'small_ball',  'l_ank_roll_link', 'r_ank_roll_link')) or
                                  sim.data.sensordata[2]> 0.6):
            fall = True
            break
        old_left_contact = left_contact
        old_right_contact = right_contact
    if mode == 'test': glfw.destroy_window(viewer.window)
    sup_cent = [0.5 * (sim.data.sensordata[30] + sim.data.sensordata[33]),
                0.5 * (sim.data.sensordata[31] + sim.data.sensordata[34])]
    COM_xyz = sim.data.sensordata[:3]
    COM_vel = sim.data.sensordata[3:6]
    sum_contact_forces = np.array([sum(old_left_contact), 
                                   sum(old_right_contact)])
    new_obs = np.concatenate((sim_state.qpos[0:17], sim_state.qvel[0:16],
                              sum_contact_forces))
    '''new_obs = np.concatenate((sim_state.qpos[7:17], sim_state.qvel[6:16],
                             [sim_state.qpos[3]]))'''
    quaternion = [sim.data.qpos[3], sim.data.qpos[4],
                  sim.data.qpos[5], sim.data.qpos[6]]
    return new_obs, COM_xyz, COM_vel, quaternion, sup_cent, fall

## rollout

In [10]:
def rollout(model, pid_time, kp, kd, ki, actor, cov_mat, gamma, rew_coef, distribution,
            timesteps_per_batch, max_timesteps_per_episode, action_range,
            device, mode, ball, obs_dim):
    batch_obs = []
    batch_acts = []
    batch_log_probs = []
    batch_rews = []
    batch_rtgs = []
    batch_lens = []
    sum_ep_rew = []
    t = 0
    while t < timesteps_per_batch:
        ep_rews = [] # rewards collected per episode

        sim = MjSim(model) 
        sim = initial_condition(sim, model, kp, kd, ki, mode, ball)
        left_contact, right_contact = contact_force(sim)
        sum_contact_forces = np.array([sum(left_contact), sum(right_contact)])
        obs = np.concatenate((sim.data.qpos[0:17], sim.data.qvel[0:16], sum_contact_forces))
    
        done = False
        ep_length = 0
        for ep_t in range(max_timesteps_per_episode):
            t += 1 
            ep_length += 1

            batch_obs.append(obs)
            action, log_prob = get_action(actor, obs, cov_mat, distribution, action_range)
            #q_des = np.concatenate((action, action))
            q_des = action + obs[7:17]
            obs, COM_xyz, COM_vel, quat, sup_cent, fall = \
            pid_step(sim, model, pid_time, kp, kd, ki, q_des, state_range, mode)
            if fall or ep_length == max_timesteps_per_episode: done = True 
            rew = get_reward(rew_coef, COM_xyz, COM_vel, quat, sup_cent, fall) * (1 - done)

            ep_rews.append(rew)
            batch_acts.append(action)
            batch_log_probs.append(log_prob)

            if done: break

        sum_ep_rew.append(np.sum(ep_rews))
        batch_lens.append(ep_t + 1)
        batch_rews.append(ep_rews)
    batch_obs = torch.tensor(np.array(batch_obs), dtype=torch.float).to(device)
    batch_acts = torch.tensor(np.array(batch_acts), dtype=torch.float).to(device)
    batch_log_probs = torch.tensor(batch_log_probs, dtype=torch.float).to(device)
    batch_rtgs = compute_rtgs(batch_rews, gamma, device)                                                              # ALG STEP 4
    return batch_obs, batch_acts, batch_log_probs, batch_rtgs, batch_lens, batch_rews, sum_ep_rew


## model and parameters

In [11]:
distribution = input('Specify the distribution type. Available choices are Normal and Bete \n')
while distribution not in ('Normal', 'Beta'):
    print('Undefined', distribution, 'distribution \n')
    distribution = input('Specify the distribution type. Available choices are Normal and Bete \n')
mj_path = mujoco_py.utils.discover_mujoco()
ball = False
xml_path = os.path.join(mj_path, 'model', 'aiutman_lower_rollpitch.xml')

model = load_model_from_path(xml_path)
sim = MjSim(model)
pid_time = 0.08

control_set = {#              [Kp,    Kd,     ki,    ac_min, ac_max, pos_min, pos_max]
               "l_hip_roll":  [40,    0.1,   15,    -0.05,   0.05,   -0.50,    0.50], #0
               "l_hip_pitch": [80,    0.1,   15,    -0.05,   0.05,   -1.57,    1.57], #1
               "l_knee":      [80,    0.1,   15,    -0.05,   0.05,    0.00,    2.80], #2
               "l_ank_pitch": [80,    0.1,   15,    -0.05,   0.05,   -1.40,    1.40], #3
               "l_ank_roll":  [80,    0.1,   15,    -0.05,   0.05,   -0.50,    0.50], #4
               "r_hip_roll":  [40,    0.1,   15,    -0.05,   0.05,   -0.50,    0.50], #5
               "r_hip_pitch": [80,    0.1,   15,    -0.05,   0.05,   -1.57,    1.57], #6
               "r_knee":      [80,    0.1,   15,    -0.05,   0.05,    0.00,    2.80], #7
               "r_ank_pitch": [80,    0.1,   15,    -0.05,   0.05,   -1.40,    1.40], #8
               "r_ank_roll":  [80,    0.1,   15,    -0.05,   0.05,   -0.50,    0.50], #9
               }

kp = np.array(list(control_set.values()))[: , 0]
kd = np.array(list(control_set.values()))[: , 1]
ki = np.array(list(control_set.values()))[: , 2]
act_minrange = np.array(list(control_set.values()))[: , 3]
act_maxrange = np.array(list(control_set.values()))[: , 4]
pos_minrange = np.array(list(control_set.values()))[: , 5]
pos_maxrange = np.array(list(control_set.values()))[: , 6]
action_range = [list(act_minrange), list(act_maxrange)]
state_range = [list(pos_minrange), list(pos_maxrange)]

mass = 5.2       # total mass exept feet
mass_f = 0.4        # feet mass
lx = 0.208            # foot length
ly = 0.132            # foot width
r_pendulum = 0.43    # pendulum length
g = 9.81              # gravity constant

epsilon = 1e-5

e_xCOM_max = lx / 2;
e_yCOM_max = ly;
e_zCOM_max = 0.3;

thetadot_crt = np.sqrt(g*(mass + mass_f)/(mass * r_pendulum))
e_vCOM_max = r_pendulum * thetadot_crt
e_quat_max = np.pi/6;

alpha_xCOM = -np.log(epsilon)/(e_xCOM_max ** 2)
alpha_yCOM = -np.log(epsilon)/(e_yCOM_max ** 2)
alpha_zCOM = -np.log(epsilon)/(e_zCOM_max ** 2)
alpha_vCOM = -np.log(epsilon)/(e_vCOM_max ** 2)
alpha_quat = -np.log(epsilon)/(e_quat_max ** 2)
rew_coef = [alpha_xCOM, alpha_yCOM, alpha_zCOM, alpha_vCOM, alpha_quat]

state_dim = 2*(len(model.joint_names) - 1) - 2
obs_dim = state_dim + 15
if distribution == 'Normal': act_dim = int(state_dim / 2)
elif distribution == 'Beta': act_dim = 2*int(state_dim / 2)

actor = PolicyNN(obs_dim, act_dim, device, distribution).to(device)
critic = PolicyNN(obs_dim, 1, device, distribution).to(device)

learning_rate = 0.00005
actor_optim = Adam(actor.parameters(), lr=learning_rate)
critic_optim = Adam(critic.parameters(), lr=learning_rate)

cov_var = torch.full(size=(act_dim,), fill_value=0.01)
cov_mat = torch.diag(cov_var).to(device)

max_timesteps_per_episode = 100
timesteps_per_batch = 10 * max_timesteps_per_episode
total_timesteps = 100000000
Epochs = 5
clip = 0.2
gamma = 0.99
entropy_beta = 0.01

Specify the distribution type. Available choices are Normal and Bete 
 Beta


In [ ]:
actor.load_state_dict(torch.load('act_rollpitch_2layer256_epoch5_lr50_baseobsAndSUMcontact'))
critic.load_state_dict(torch.load('crt_rollpitch_2layer256_epoch5_lr50_baseobsAndSUMcontact'))

## train

In [ ]:
t_so_far = 0 # Timesteps simulated so far
i_so_far = 0 # Iterations ran so far
last_rewards = deque(maxlen = 5)
max_mean_reward = 0
scores = []
#with open("scores_rollpitch_2layer256_epoch5_lr50_baseobsAndSUMcontact", "rb") as fp:  scores = pickle.load(fp)
    
while t_so_far < total_timesteps:                                                      
    batch_obs, batch_acts, batch_log_probs, batch_rtgs, batch_lens, batch_rews, sum_ep_rew = \
    rollout(model, pid_time, kp, kd, ki, actor, cov_mat, gamma, rew_coef, distribution,
            timesteps_per_batch, max_timesteps_per_episode, action_range,
            device, 'train', ball, obs_dim)
    
    t_so_far += np.sum(batch_lens)
    i_so_far += 1

    V, _ = evaluate(actor, critic, cov_mat, batch_obs, batch_acts, distribution, action_range)
    A_k = batch_rtgs - V.detach()                                                                 
    A_k = (A_k - A_k.mean()) / (A_k.std() + 1e-10)

    # This is the loop where we update our network for some n epochs
    for epoch in range(Epochs):                                                       
        # Calculate V_phi and pi_theta(a_t | s_t)
        V, curr_log_probs = evaluate(actor, critic, cov_mat, batch_obs, batch_acts,
                                     distribution, action_range)
        ratios = torch.exp(curr_log_probs - batch_log_probs)

        # Calculate surrogate losses.
        surr1 = ratios * A_k
        surr2 = torch.clamp(ratios, 1 - clip, 1 + clip) * A_k
 
        entropy = -(batch_acts * batch_log_probs.reshape(batch_acts.shape[0],-1)
                   ).sum(dim=1).mean()
        loss_e = -entropy_beta * entropy
        
        actor_loss = (-torch.min(surr1, surr2)).mean().to(device)# + loss_e
        critic_loss = nn.MSELoss()(V, batch_rtgs)# + loss_e

        actor_optim.zero_grad()
        actor_loss.backward(retain_graph=True)
        actor_optim.step()

        critic_optim.zero_grad()
        critic_loss.backward()
        critic_optim.step()
    
    last_rewards.append(np.mean(sum_ep_rew))
    scores.append(np.mean(sum_ep_rew))

    print("In iteratin", i_so_far, "mean ep length is:", "{:.2f}".format(np.mean(batch_lens)),
          'and mean ep rewards is:', "{:.2f}".format(np.mean(sum_ep_rew)))
    if np.mean(sum_ep_rew) > max_mean_reward:
        os.system('play -nq -t alsa synth {} sine {}'.format(0.25, 440))
        print('maximum mean reward is acheived!')
        torch.save(actor.state_dict(), 'act_rollpitch_2layer256_epoch5_lr50_baseobsAndSUMcontact')
        torch.save(critic.state_dict(), 'crt_rollpitch_2layer256_epoch5_lr50_baseobsAndSUMcontact')
        with open("scores_rollpitch_2layer256_epoch5_lr50_baseobsAndSUMcontact", "wb") as fp: pickle.dump(scores, fp)
        max_mean_reward = np.mean(sum_ep_rew)
        
    if np.mean(last_rewards) > 0.92*max_timesteps_per_episode or np.mean(sum_ep_rew)<0.0001:
        torch.save(actor.state_dict(), 'act_rollpitch_2layer256_epoch5_lr50_baseobsAndSUMcontact')
        torch.save(critic.state_dict(), 'crt_rollpitch_2layer256_epoch5_lr50_baseobsAndSUMcontact')       
        os.system('play -nq -t alsa synth {} sine {}'.format(3, 440))
        print("solved!")
        with open("scores_rollpitch_2layer256_epoch5_lr50_baseobsAndSUMcontact", "wb") as fp: pickle.dump(scores, fp)
        break

reg = LinearRegression().fit(np.arange(len(scores)).reshape(-1, 1),
                             np.array(scores).reshape(-1, 1))
score_pred = reg.predict(np.arange(len(scores)).reshape(-1, 1))
plt.plot(scores)
plt.ylabel('rewards')
plt.xlabel('iterations')
plt.plot(score_pred)

In iteratin 1 mean ep length is: 2.48 and mean ep rewards is: 1.20
maximum mean reward is acheived!
In iteratin 2 mean ep length is: 2.27 and mean ep rewards is: 1.06
In iteratin 3 mean ep length is: 2.63 and mean ep rewards is: 1.32
maximum mean reward is acheived!
In iteratin 4 mean ep length is: 2.64 and mean ep rewards is: 1.33
maximum mean reward is acheived!
In iteratin 5 mean ep length is: 2.71 and mean ep rewards is: 1.34
maximum mean reward is acheived!
In iteratin 6 mean ep length is: 2.69 and mean ep rewards is: 1.33
In iteratin 7 mean ep length is: 2.75 and mean ep rewards is: 1.37
maximum mean reward is acheived!
In iteratin 8 mean ep length is: 2.81 and mean ep rewards is: 1.43
maximum mean reward is acheived!
In iteratin 9 mean ep length is: 3.03 and mean ep rewards is: 1.59
maximum mean reward is acheived!
In iteratin 10 mean ep length is: 3.59 and mean ep rewards is: 2.00
maximum mean reward is acheived!
In iteratin 11 mean ep length is: 3.02 and mean ep rewards is: 1.

In [ ]:
torch.save(actor.state_dict(), 'act_rollpitch_2layer256_epoch5_lr50_baseobsAndcontact')
torch.save(critic.state_dict(), 'crt_rollpitch_2layer256_epoch5_lr50_baseobsAndcontact')


In [ ]:
with open("scores_rollpitch_2layer256_epoch5_contfeet2", "rb") as fp:  scores = pickle.load(fp)
reg = LinearRegression().fit(np.arange(len(scores)).reshape(-1, 1),
                             np.array(scores).reshape(-1, 1))
score_pred = reg.predict(np.arange(len(scores)).reshape(-1, 1))

plt.plot(scores)
plt.ylabel('mean rewards')
plt.xlabel('iterations')
plt.plot(score_pred)

In [ ]:
with open("scores_rollpitch_2layer256_epoch5", "rb") as fp:  scores = pickle.load(fp)
reg = LinearRegression().fit(np.arange(len(scores)).reshape(-1, 1),
                             np.array(scores).reshape(-1, 1))
score_pred = reg.predict(np.arange(len(scores)).reshape(-1, 1))

plt.plot(scores)
plt.ylabel('mean rewards')
plt.xlabel('iterations')
plt.plot(score_pred)

In [ ]:
for _ in range(10):
    sim = MjSim(model)
    sim = initial_condition(sim, model, kp, kd, ki, state_range, 'test', ball)

In [ ]:
mj_path = mujoco_py.utils.discover_mujoco()
xml_path = os.path.join(mj_path, 'model', 'aiutman_lower_rollpitch.xml')
model = load_model_from_path(xml_path)
sim = MjSim(model)

In [ ]:
viewer = MjViewer(sim)
c_array = np.zeros(6, dtype=np.float64)
for ii in range(200):
    sim.data.xfrc_applied[1][1]  = 10
    sim.step()
    #print('\n number of contacts', sim.data.ncon)
    for i in range(sim.data.ncon):
        #print(i)
        contact = sim.data.contact[i]
        #geom1_body = sim.model.geom_bodyid[contact.geom1]
        #geom2_body = sim.model.geom_bodyid[contact.geom2]
        #print('geom1', contact.geom1, sim.model.geom_id2name(contact.geom1),'body', geom1_body)
        #print('geom2', contact.geom2, sim.model.geom_id2name(contact.geom2),'body', geom2_body)
        print()
        if contact.geom2 in [10, 19]:
            print(i)
            #print('geom1', contact.geom1)
            print('geom2', contact.geom2)
            # Use internal functions to read out mj_contactForce
            mujoco_py.functions.mj_contactForce(sim.model, sim.data, i, c_array)
            print('c_array', c_array[0])
            print(sim.data.sensordata[8], sim.data.sensordata[11],
                  sim.data.sensordata[14], sim.data.sensordata[17])
            print(sim.data.sensordata[20], sim.data.sensordata[23],
                  sim.data.sensordata[26], sim.data.sensordata[29])
            
    contact_forces = contact_force(sim)
    print(contact_forces)
    viewer.render()
    time.sleep(0.01)
glfw.destroy_window(viewer.window)